# TIỀN XỬ LÍ DỮ LIỆU

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv("../datasets/historical_air_quality_2021_en.csv")
df.head()

,Station ID,AQI index,Location,Station name,Url,Dominent pollutant,CO,Dew,Humidity,NO2,...,Pressure,PM10,PM2.5,SO2,Temperature,Wind,Data Time S,Data Time Tz,Status,Alert level
0,8767.0,102.00,"10.782978,106.700711","Ho Chi Minh City US Consulate, Vietnam (Lãnh s...",https://aqicn.org/city/vietnam/ho-chi-minh-cit...,pm25,-,NaN,83.0,-,...,"1,009.00",NaN,102.00,-,27.0,3.6,2021-01-21 19:00:00,+07:00,#NAME?,#NAME?
1,8688.0,221.00,"21.0811211,105.8180306","United Nations International School of Hanoi, ...",https://aqicn.org/city/vietnam/hanoi/unis,aqi,-,NaN,77.0,-,...,"1,015.00",NaN,-,-,18.0,1.5,2021-01-21 20:00:00,+07:00,#NAME?,#NAME?
2,8641.0,281.00,"21.0215063,105.8188748","Hanoi US Embassy, Vietnam (Đại sứ quán Mỹ, Hà ...",https://aqicn.org/city/vietnam/hanoi/us-embassy,pm25,0.30,NaN,77.0,8.70,...,"1,015.00",NaN,281.00,2.10,18.0,1.5,2021-01-21 20:00:00,+07:00,#NAME?,#NAME?
3,13012.0,36.00,"13.998599,107.996482","Gia Lai/phường Thống Nhất - Pleiku, Vietnam",https://aqicn.org/city/vietnam/gia-lai/phuong-...,pm25,3.00,NaN,60.2,6.00,...,923.70,29.00,36.00,1.00,24.3,1.0,2021-01-21 20:00:00,+07:00,#NAME?,#NAME?
4,12488.0,68.00,"16.46226,107.596351","Thừa Thiên Huế/83 Hùng Vương, Vietnam",https://aqicn.org/city/vietnam/thua-thien-hue/...,pm25,2.00,NaN,88.0,-,...,"1,015.00",52.00,68.00,-,21.0,1.0,2021-01-21 19:00:00,+07:00,#NAME?,#NAME?


## Dữ liệu có bao nhiêu dòng, bao nhiêu cột?

In [3]:
num_rows, num_cols = df.shape
num_rows, num_cols

(3415, 21)

Loại bỏ những cột không cần thiết

Ý nghĩa các cột không cần thiết:
- Station ID: ID của trạm quan trắc không khí.
- Url: Đường dẫn đến trang web liên quan đến dữ liệu không khí của trạm.
- Status: Trạng thái dữ liệu.
- Alert level: Cấp độ cảnh báo.
- Data Time Tz: Múi giờ của dữ liệu.


In [4]:
to_be_droped = ['Station ID', 'Url','Status', 'Alert level', 'Data Time Tz']
df = df.drop(columns=to_be_droped)

In [5]:
df = df.replace('-', np.nan)
df.isna().sum()

AQI index             1314
Location               793
Station name           793
Dominent pollutant    1301
CO                    1260
Dew                   1152
Humidity               807
NO2                   1423
O3                    2188
Pressure               807
PM10                  1527
PM2.5                 1231
SO2                   2154
Temperature            800
Wind                   800
Data Time S            793
dtype: int64

Quan sát thấy có số lượng lớn dữ liệu bị thiếu và có những cột phải xử lý kiểu dữ liệu do vậy ta phải tiền xử lý dữ liệu.

## Tiền xử lí dữ liệu

Đưa cột `Data Time S` về kiểu dữ liệu `datetime`

In [6]:
def cast_to_datetime(datetime_str):
    res = pd.to_datetime(datetime_str)

    if res.tzinfo is not None:
        res = res.tz_localize(None)

    return res

df['Data Time S'] = df['Data Time S'].apply(cast_to_datetime)

Đưa các cột `numerical` về đúng kiểu dữ liệu

In [7]:
df['Pressure'] = df['Pressure'].str.replace(',', '')

numerical_labels = ['AQI index', 'CO', 'Dew',
                    'Humidity', 'NO2', 'O3', 'Pressure',
                    'PM10', 'PM2.5', 'SO2', 'Temperature', 'Wind']
df[numerical_labels] = df[numerical_labels].astype('float64')

Gom nhóm các tỉnh thành

In [8]:
state_labels = ["Hà Nội", "Bắc Ninh", "Quảng Ninh", "Cao Bằng", "Gia Lai",
                "Lào Cai", "Nha Trang", "Hồ Chí Minh", "Đà Nẵng", "Thừa Thiên Huế", "Hạ Long"]


def classify_region(station_name):
    if (not isinstance(station_name, str)):
        return station_name

    for state in state_labels:
        if state in station_name:
            return state
    print(station_name)


df['Station name'] = df['Station name'].apply(classify_region)


Điền các cột dữ liệu còn thiếu bằng trung vị của khu vực đó

In [9]:
def fill_missing_value(x):
    same_station = df[df["Station name"] == x["Station name"]]
    for col in numerical_labels:
        if col == "AQI index":
            continue
        if np.isnan(x[col]):
            if same_station[col].isna().all():
                x[col] = df[col].median()
            else:
                x[col] = same_station[col].median()
    return x
df = df.apply(fill_missing_value, axis=1)

Điền giá trị cho các dòng không có `AQI index`, `Dominent pollutant` theo công thức tính AQI của Cục Bảo vệ Môi trường Hoa Kỳ (EPA)
https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf

In [10]:
# Define breakpoints and AQI levels
breakpoints = {
    'O3': [(0, 54), (55, 70), (71, 85), (86, 105), (106, 200)],
    'PM2.5': [(0.0, 12.0), (12.1, 35.4), (35.5, 55.4), (55.5, 150.4), (150.5, 250.4), (250.5, 350.4), (350.5, 500.4)],
    'PM10': [(0, 54), (55, 154), (155, 254), (255, 354), (355, 424), (425, 504), (505, 604)],
    'CO': [(0.0, 4.4), (4.5, 9.4), (9.5, 12.4), (12.5, 15.4), (15.5, 30.4), (30.5, 40.4), (40.5, 50.4)],
    'SO2': [(0, 35), (36, 75), (76, 185), (186, 304)],
    'NO2': [(0, 53), (54, 100), (101, 360), (361, 649), (650, 1249), (1250, 1649), (1650, 2049)]
}
aqi_levels = [(0, 50), (51, 100), (101, 150), (151, 200),
              (201, 300), (301, 400), (401, 500)]


# Define function to calculate AQI base on formula
def aqi_formula(concentration, c, i):
    aqi = ((concentration - c[0]) / (c[1] - c[0])) * \
        (i[1] - i[0]) + i[0]
    return aqi


# Define function to calculate AQI for a single pollutant


def calculate_individual_aqi(pollutant_name, concentration):
    if math.isnan(concentration):
        return np.nan
    bps = breakpoints[pollutant_name]

    for i in range(len(bps)):
        if bps[i][0] <= concentration <= bps[i][1]:
            aqi = aqi_formula(concentration, bps[i], aqi_levels[i])
            return round(aqi)
    last_level = len(bps) - 1
    return round(aqi_formula(concentration, bps[last_level], aqi_levels[last_level]))


def calculate_aqi(pollutant_concentrations):

    # Calculate AQI for each pollutant
    AQI_indexes = [calculate_individual_aqi(
        pollutant, pollutant_concentrations[pollutant]) for pollutant in breakpoints.keys()]

    # Return maximum AQI value
    if np.isnan(AQI_indexes).all():
        return np.nan
    else:
        return max(AQI_indexes, key=lambda index: 0 if np.isnan(index) else index)


def specify_dominant_pollutant(pollutant_concentrations):
    pollutant_aqi_dict = {pollutant: calculate_individual_aqi(
        pollutant, pollutant_concentrations[pollutant]) for pollutant in breakpoints.keys()}

    return max(breakpoints.keys(), key=lambda pollutant: 0 if np.isnan(pollutant_aqi_dict[pollutant]) else pollutant_aqi_dict[pollutant])


aqi_na_rows = df['AQI index'].isna()
df.loc[aqi_na_rows, 'AQI index'] = df[aqi_na_rows].apply(calculate_aqi, axis=1)

dominant_pollu_trans_dict = {'pm25': 'PM2.5', 'aqi': 'aqi', 'pm10': 'PM10'}
dominant_na_rows = df['Dominent pollutant'].isna()
df.loc[~dominant_na_rows, 'Dominent pollutant'] = df[~dominant_na_rows].apply(lambda x:
                                                                              dominant_pollu_trans_dict[x['Dominent pollutant']], axis=1)
df.loc[dominant_na_rows, 'Dominent pollutant'] = df[dominant_na_rows].apply(
    specify_dominant_pollutant, axis=1)


Xây dựng lại cột `Status` theo tiêu chuẩn của Cục Bảo vệ Môi trường Hoa Kỳ (US-EPA)

|AQI|Status|
|--|:------:|
|0-50|Good|
|51-100|Moderate|
|101-150|Unhealthy for sensitive groups|
|151-200|Unhealthy|
|201-300|Very unhealthy|
|301+|Hazardous|

In [11]:
def status(x):
    if 0 <= x <= 50:
        return 'Good'
    if 51 <= x <= 100:
        return 'Moderate'
    if 101 <= x <= 150:
        return 'Unhealthy for sensitive groups'
    if 151 <= x <= 200:
        return 'Unhealthy'
    if 201 <= x <= 300:
        return 'Very unhealthy'
    if 301 <= x:
        return 'Hazardous'
df['Status'] = df['AQI index'].apply(status)

Xóa những dòng trùng lặp / thiếu dữ liệu

In [12]:
df = df.dropna()
df = df.drop_duplicates()

Tách `Location` thành `Latitude` và `Longitude`

In [13]:
location = df['Location'].str.split(',')
df['Latitude'] = location.apply(lambda x: x[0]).astype(float)
df['Longitude'] = location.apply(lambda x: x[1]).astype(float)
df = df.drop(columns='Location')

#### Lưu dataset đã xử lí

In [14]:
# df.to_csv("./processed.csv", index=False)

## Xử lý dữ liệu khu vực Hà Nội
https://aqicn.org/data-platform

In [15]:
df_HN=pd.read_csv("../datasets/hanoi-air-quality.csv")
df_HN.date =df_HN.date.apply(pd.to_datetime)
df_HN.set_index('date', inplace=True)
df_HN.sort_index(inplace=True)
df_HN

,pm25,pm10,o3,no2,so2,co
date,,,,,,
2014-01-01,,124,22,40,6,28
2014-01-02,186,144,69,39,8,43
2014-01-03,227,151,24,25,12,28
2014-01-04,253,,,,,
2014-01-05,,115,65,35,5,33
...,...,...,...,...,...,...
2023-07-08,35,39,,26,19,6
2023-07-09,30,46,,28,19,7
2023-07-10,46,44,16,27,19,7


In [16]:
df_HN=df_HN.loc['2020-01-01':'2023-07-11']
df_HN

,pm25,pm10,o3,no2,so2,co
date,,,,,,
2020-01-01,153,65,5,25,4,15
2020-01-02,118,44,6,21,3,10
2020-01-03,59,44,6,21,3,12
2020-01-04,70,46,6,18,2,9
2020-01-05,66,47,7,17,2,9
...,...,...,...,...,...,...
2023-07-07,34,36,,18,19,5
2023-07-08,35,39,,26,19,6
2023-07-09,30,46,,28,19,7


In [17]:
df_HN = df_HN.replace(' ', np.nan)

In [18]:
df_HN.rename(columns={' co':'CO',
      ' no2':'NO2', ' o3':'O3', ' pm10':'PM10',
       ' pm25':'PM2.5',' so2':'SO2',
       }, inplace=True)

In [19]:
numerical_labels = ['CO','NO2', 'O3', 
                    'PM10', 'PM2.5', 'SO2']
df_HN[numerical_labels] = df_HN[numerical_labels].astype('float64')

In [20]:
df_HN.resample('M').mean()

,PM2.5,PM10,O3,NO2,SO2,CO
date,,,,,,
2020-01-31,79.967742,46.032258,8.870968,15.419355,3.483871,9.516129
2020-02-29,91.068966,48.551724,9.689655,18.800000,4.965517,10.379310
2020-03-31,57.586207,33.206897,14.413793,13.827586,4.320000,8.758621
2020-04-30,41.133333,23.866667,19.166667,5.433333,7.900000,6.300000
2020-05-31,39.000000,33.387097,18.516129,6.111111,5.774194,8.612903
2020-06-30,30.300000,32.033333,18.900000,12.833333,4.066667,7.766667
2020-07-31,30.096774,29.129032,18.741935,11.000000,2.548387,7.903226
2020-08-31,27.161290,21.419355,11.187500,11.322581,3.838710,11.000000
2020-09-30,32.366667,23.366667,NaN,11.900000,4.866667,9.833333


In [21]:
monthly_mean = df_HN.resample('M').mean()
monthly_mean = monthly_mean.fillna(method="ffill")
monthly_mean

,PM2.5,PM10,O3,NO2,SO2,CO
date,,,,,,
2020-01-31,79.967742,46.032258,8.870968,15.419355,3.483871,9.516129
2020-02-29,91.068966,48.551724,9.689655,18.800000,4.965517,10.379310
2020-03-31,57.586207,33.206897,14.413793,13.827586,4.320000,8.758621
2020-04-30,41.133333,23.866667,19.166667,5.433333,7.900000,6.300000
2020-05-31,39.000000,33.387097,18.516129,6.111111,5.774194,8.612903
2020-06-30,30.300000,32.033333,18.900000,12.833333,4.066667,7.766667
2020-07-31,30.096774,29.129032,18.741935,11.000000,2.548387,7.903226
2020-08-31,27.161290,21.419355,11.187500,11.322581,3.838710,11.000000
2020-09-30,32.366667,23.366667,11.187500,11.900000,4.866667,9.833333


In [22]:
df_HN = df_HN.asfreq('D')
df_HN.isna().sum()

PM2.5     50
PM10      49
O3       148
NO2       67
SO2       53
CO        48
dtype: int64

In [23]:
for index, row in df_HN.iterrows():
    year = index.year
    month = index.month

    # Xác định ngày cuối cùng của tháng trong df_HN
    last_day_of_month = pd.Timestamp(year, month, 1) + pd.offsets.MonthEnd()

    # Kiểm tra ngày cuối cùng của tháng có trong chỉ mục của monthly_mean hay không
    if last_day_of_month in monthly_mean.index:
        mean_row = monthly_mean.loc[last_day_of_month]

        # Lặp qua các cột và điền giá trị NaN
        for col in df_HN.columns:
            if pd.isna(row[col]):
                df_HN.at[index, col] = mean_row[col]


df_HN.isna().sum()

PM2.5    0
PM10     0
O3       0
NO2      0
SO2      0
CO       0
dtype: int64

In [24]:
df_HN

,PM2.5,PM10,O3,NO2,SO2,CO
date,,,,,,
2020-01-01,153.0,65.0,5.0,25.0,4.0,15.0
2020-01-02,118.0,44.0,6.0,21.0,3.0,10.0
2020-01-03,59.0,44.0,6.0,21.0,3.0,12.0
2020-01-04,70.0,46.0,6.0,18.0,2.0,9.0
2020-01-05,66.0,47.0,7.0,17.0,2.0,9.0
...,...,...,...,...,...,...
2023-07-07,34.0,36.0,13.0,18.0,19.0,5.0
2023-07-08,35.0,39.0,13.0,26.0,19.0,6.0
2023-07-09,30.0,46.0,13.0,28.0,19.0,7.0


In [25]:
df_HN['AQI'] = np.full(df_HN.shape[0], np.nan)

In [26]:
# Đánh dấu các dòng có giá trị AQI không khả dụng và tính toán AQI cho chúng
aqi_na_rows = df_HN['AQI'].isna()
df_HN.loc[aqi_na_rows, 'AQI'] = df_HN[aqi_na_rows].apply(calculate_aqi, axis=1)
df_HN.AQI

date
2020-01-01    203.0
2020-01-02    183.0
2020-01-03    153.0
2020-01-04    158.0
2020-01-05    156.0
              ...  
2023-07-07     97.0
2023-07-08     99.0
2023-07-09     89.0
2023-07-10    127.0
2023-07-11    132.0
Freq: D, Name: AQI, Length: 1288, dtype: float64

In [27]:
df_HN

,PM2.5,PM10,O3,NO2,SO2,CO,AQI
date,,,,,,,
2020-01-01,153.0,65.0,5.0,25.0,4.0,15.0,203.0
2020-01-02,118.0,44.0,6.0,21.0,3.0,10.0,183.0
2020-01-03,59.0,44.0,6.0,21.0,3.0,12.0,153.0
2020-01-04,70.0,46.0,6.0,18.0,2.0,9.0,158.0
2020-01-05,66.0,47.0,7.0,17.0,2.0,9.0,156.0
...,...,...,...,...,...,...,...
2023-07-07,34.0,36.0,13.0,18.0,19.0,5.0,97.0
2023-07-08,35.0,39.0,13.0,26.0,19.0,6.0,99.0
2023-07-09,30.0,46.0,13.0,28.0,19.0,7.0,89.0


In [28]:
# df_HN.to_csv("data_HN_2020_2023.csv")